In [1]:
import os
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.random import set_seed

In [ ]:
### Importación de los datos

In [2]:
# Leer el archivo CSV
df = pd.read_csv('diabetic_data.csv')

# Mostrar las primeras filas del dataset
print(df.head())

# Resumen de la información del dataset
print(df.info())

# Mostrar la cantidad de valores nulos por columna
print(df.isnull().sum())

   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No

In [3]:
# Leer el archivo CSV, tratando '?' como valores nulos
df = pd.read_csv('diabetic_data.csv', na_values='?')

# Mostrar la cantidad de valores nulos por columna
print(df.isnull().sum())

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv('diabetic_data_modified.csv', index=False)

print("El archivo modificado ha sido guardado como 'diabetic_data_modified.csv'.")


C:\Users\grego\AppData\Local\Temp\ipykernel_18280\4281049621.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('diabetic_data.csv', na_values='?')


encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [ ]:
### Procesamiento de los datos

In [4]:
# Cargar el dataset
df = pd.read_csv('diabetic_data_modified.csv')

# Asegurarse de manejar correctamente los valores nulos
# Identificar las columnas con valores nulos según la descripción proporcionada
cols_with_missing = ['race', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']

# Imputar los valores nulos con la moda para variables categóricas
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[cols_with_missing] = categorical_imputer.fit_transform(df[cols_with_missing])

# Limpiar y convertir 'weight' a valores numéricos y luego categorizarlo
def clean_weight(weight_str):
    if weight_str == "?":
        return None
    elif weight_str.startswith(">"):
        return float(weight_str[1:]) + 1  # Incrementar en 1 para asegurar que los límites sean correctos
    elif weight_str.startswith("["):
        return float(weight_str.strip("[]").split("-")[0])
    elif weight_str == "Unknown":
        return None
    else:
        return float(weight_str)

df['weight'] = df['weight'].apply(clean_weight)

# Definir los rangos de peso y codificar 'weight'
weight_ranges = ['[0-25)', '[25-50)', '[50-75)', '[75-100)', '[100-125)', '[125-150)', '[150-175)', '[175-200)', '>200']
df['weight_category'] = pd.cut(df['weight'], bins=[0, 25, 50, 75, 100, 125, 150, 175, 200, float('inf')], labels=weight_ranges, right=False)

# Eliminar la columna original 'weight'
df.drop(columns=['weight'], inplace=True)

# Función para asignar categorías a los códigos ICD-9
def assign_icd_category(icd_code):
    if pd.isnull(icd_code):
        return 'Unknown'
    if icd_code.startswith(('E', 'V')):
        return 'E-V codes'
    else:
        code_number = int(icd_code.split('.')[0])  # Tomar solo el número de código ICD-9
        if 1 <= code_number <= 139:
            return '001-139'
        elif 140 <= code_number <= 239:
            return '140-239'
        elif 240 <= code_number <= 279:
            return '240-279'
        elif 280 <= code_number <= 289:
            return '280-289'
        elif 290 <= code_number <= 319:
            return '290-319'
        elif 320 <= code_number <= 389:
            return '320-389'
        elif 390 <= code_number <= 459:
            return '390-459'
        elif 460 <= code_number <= 519:
            return '460-519'
        elif 520 <= code_number <= 579:
            return '520-579'
        elif 580 <= code_number <= 629:
            return '580-629'
        elif 630 <= code_number <= 679:
            return '630-679'
        elif 680 <= code_number <= 709:
            return '680-709'
        elif 710 <= code_number <= 739:
            return '710-739'
        elif 740 <= code_number <= 759:
            return '740-759'
        elif 760 <= code_number <= 779:
            return '760-779'
        elif 780 <= code_number <= 799:
            return '780-799'
        elif 800 <= code_number <= 999:
            return '800-999'
        else:
            return 'Other'  # En caso de no encontrar una categoría válida

# Aplicar la función a cada columna de diagnóstico
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[col + '_category'] = df[col].apply(assign_icd_category)

# Eliminar las columnas originales de diagnóstico
df.drop(columns=['diag_1', 'diag_2', 'diag_3'], inplace=True)

# Función para asignar valores únicos a las franjas de edad
def age_to_value(age_str):
    age_mapping = {
        '[0-10)': 5,
        '[10-20)': 15,
        '[20-30)': 25,
        '[30-40)': 35,
        '[40-50)': 45,
        '[50-60)': 55,
        '[60-70)': 65,
        '[70-80)': 75,
        '[80-90)': 85,
        '[90-100)': 95
    }
    return age_mapping.get(age_str, None)

# Aplicar la función de agrupamiento de edades
df['age'] = df['age'].apply(age_to_value)

# Aplicar codificación one-hot a las variables categóricas, excluyendo 'age' ya que está mapeada a valores únicos
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'age' in categorical_cols:
    categorical_cols.remove('age')
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_cols = pd.DataFrame(encoder.fit_transform(df[categorical_cols]))

# Sustituir las columnas originales con las nuevas codificadas
encoded_cols.columns = encoder.get_feature_names_out(categorical_cols)
df.drop(columns=categorical_cols, inplace=True)
df = pd.concat([df, encoded_cols], axis=1)

# Guardar el resultado en un nuevo archivo CSV
# df.to_csv('processed_data.csv', index=False)

print("Datos procesados y guardados en 'processed_data.csv'.")

C:\Users\grego\AppData\Local\Temp\ipykernel_18280\2766882862.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('diabetic_data_modified.csv')


Datos procesados y guardados en 'processed_data.csv'.


In [ ]:
### Creación del modelo LSTM

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Función para cargar los datos
def load_data(filepath):
    return pd.read_csv(filepath)

# Función para crear el modelo LSTM
def crear_modelo_lstm(input_shape, learning_rate=0.001, dropout_rate=0.5, regularization_rate=0.01):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=input_shape, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(regularization_rate)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LSTM(64, kernel_regularizer=tf.keras.regularizers.l2(regularization_rate)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Función de entrenamiento en cada dispositivo
def train_on_device(X_train, y_train, input_shape, epochs, learning_rate, batch_size):
    model = crear_modelo_lstm(input_shape, learning_rate)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=1)
    return model

# Función para dividir los datos
def split_data(X, y, num_parts):
    X_splits = np.array_split(X, num_parts)
    y_splits = np.array_split(y, num_parts)
    return X_splits, y_splits

# Función para preprocesar datos una sola vez
def preprocess_data(X, y, num_parts):
    X_splits, y_splits = split_data(X, y, num_parts)
    scalers = [StandardScaler().fit(X_split) for X_split in X_splits]
    X_scaled_splits = [scaler.transform(X_split) for scaler, X_split in zip(scalers, X_splits)]
    X_scaled_reshaped = [X_scaled.reshape(-1, X.shape[1], 1) for X_scaled in X_scaled_splits]
    return X_scaled_reshaped, y_splits, scalers

# Función principal para la simulación de Federated Learning
def federated_learning(X, y, num_parts, num_iterations, epochs=6, learning_rate=0.001, batch_size=64):
    all_accuracies = []
    local_accuracies = []

    X_scaled_splits, y_splits, scalers = preprocess_data(X, y, num_parts)

    for iteration in range(num_iterations):
        print(f'Iteración {iteration + 1}/{num_iterations}')
        local_models = []

        for i in range(num_parts):
            print(f'Entrenando modelo local {i + 1}/{num_parts}')
            X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled_splits[i], y_splits[i], test_size=0.4, random_state=iteration, stratify=y_splits[i])
            
            local_model = train_on_device(X_train_scaled, y_train, input_shape=(X.shape[1], 1), epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)
            local_models.append(local_model)
            
            # Evaluar el modelo local en datos de prueba local
            loss_local, accuracy_local = local_model.evaluate(X_test_scaled, y_test, verbose=0)
            local_accuracies.append(accuracy_local)
            print(f'Accuracy del modelo local {i + 1} en datos de prueba: {accuracy_local:.5f}')

        # Crear el modelo global combinando los modelos locales
        print('Creando modelo global')
        global_model = crear_modelo_lstm(input_shape=(X.shape[1], 1))
        
        # Promediar los pesos capa por capa (Federated Averaging)
        global_weights = [np.mean([model.get_weights()[layer] for model in local_models], axis=0) for layer in range(len(local_models[0].get_weights()))]
        global_model.set_weights(global_weights)
        
        # Evaluar el modelo global en datos de prueba combinados
        X_test_combined = np.vstack([X_scaled_splits[i] for i in range(num_parts)])
        y_test_combined = np.hstack([y_splits[i] for i in range(num_parts)])
        
        loss_global, accuracy_global = global_model.evaluate(X_test_combined, y_test_combined, verbose=0)
        
        all_accuracies.append(accuracy_global)
    
    return all_accuracies, local_accuracies

# Cargar datos desde el archivo CSV
df = load_data('processed_data.csv')

# Seleccionar características y etiquetas
X = df.drop(columns=['diabetesMed_Yes'])
y = df['diabetesMed_Yes']

# Ejecutar la simulación de Federated Learning
accuracies_global, accuracies_local = federated_learning(X, y, num_parts=5, num_iterations=1, epochs=10, learning_rate=0.001, batch_size=64)

# Imprimir resultados
print(f'Accuracies de los modelos locales:')
for i, acc in enumerate(accuracies_local):
    print(f'Modelo local {i + 1}: {acc:.5f}')
    
print(f'Accuracy del modelo global después de la iteración: {accuracies_global[0]:.5f}')

Iteración 1/1
Entrenando modelo local 1/5


C:\Users\grego\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 118s 587ms/step - accuracy: 0.7155 - loss: 1.4090 - val_accuracy: 0.7340 - val_loss: 0.5884
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 101s 586ms/step - accuracy: 0.7421 - loss: 0.5818 - val_accuracy: 0.7340 - val_loss: 0.5795
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 93s 540ms/step - accuracy: 0.7344 - loss: 0.5827 - val_accuracy: 0.7340 - val_loss: 0.5799
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 81s 473ms/step - accuracy: 0.7332 - loss: 0.5842 - val_accuracy: 0.7340 - val_loss: 0.5793
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 85s 497ms/step - accuracy: 0.7279 - loss: 0.5900 - val_accuracy: 0.7340 - val_loss: 0.5792
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 82s 479ms/step - accuracy: 0.7362 - loss: 0.5798 - val_accuracy: 0.7340 - val_loss: 0.5821
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 86s 500ms/step - accuracy: 0.7302 - loss: 0.5859 - val_accuracy: 0.7340 - val_loss: 0.5793
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 90s 521ms/step - accuracy: 0.7321 - loss:

C:\Users\grego\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


172/172 ━━━━━━━━━━━━━━━━━━━━ 92s 510ms/step - accuracy: 0.7436 - loss: 1.3856 - val_accuracy: 0.7602 - val_loss: 0.5704
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 87s 505ms/step - accuracy: 0.7649 - loss: 0.5573 - val_accuracy: 0.7602 - val_loss: 0.5519
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 91s 528ms/step - accuracy: 0.7563 - loss: 0.5600 - val_accuracy: 0.7602 - val_loss: 0.5516
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 93s 538ms/step - accuracy: 0.7554 - loss: 0.5601 - val_accuracy: 0.7602 - val_loss: 0.5513
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 98s 571ms/step - accuracy: 0.7567 - loss: 0.5594 - val_accuracy: 0.7602 - val_loss: 0.5510
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 95s 553ms/step - accuracy: 0.7558 - loss: 0.5593 - val_accuracy: 0.7602 - val_loss: 0.5516
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 91s 527ms/step - accuracy: 0.7602 - loss: 0.5535 - val_accuracy: 0.7602 - val_loss: 0.5510
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 89s 519ms/step - accuracy: 0.7597 - loss: 0.5549 - val

...

Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 115s 559ms/step - accuracy: 0.7492 - loss: 1.3933 - val_accuracy: 0.7594 - val_loss: 0.5607
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 94s 543ms/step - accuracy: 0.7618 - loss: 0.5577 - val_accuracy: 0.7594 - val_loss: 0.5531
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 94s 547ms/step - accuracy: 0.7555 - loss: 0.5618 - val_accuracy: 0.7594 - val_loss: 0.5518
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 93s 539ms/step - accuracy: 0.7581 - loss: 0.5580 - val_accuracy: 0.7594 - val_loss: 0.5526
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 90s 526ms/step - accuracy: 0.7561 - loss: 0.5601 - val_accuracy: 0.7594 - val_loss: 0.5521
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 99s 578ms/step - accuracy: 0.7651 - loss: 0.5504 - val_accuracy: 0.7594 - val_loss: 0.5518
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 90s 522ms/step - accuracy: 0.7601 - loss: 0.5536 - val_accuracy: 0.7594 - val_loss: 0.5521
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 88s 513ms/step - accuracy: 0.7576 - loss: 

C:\Users\grego\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


172/172 ━━━━━━━━━━━━━━━━━━━━ 102s 566ms/step - accuracy: 0.7928 - loss: 1.3469 - val_accuracy: 0.8044 - val_loss: 0.5038
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 90s 524ms/step - accuracy: 0.8063 - loss: 0.4998 - val_accuracy: 0.8044 - val_loss: 0.4947
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 88s 511ms/step - accuracy: 0.8097 - loss: 0.4925 - val_accuracy: 0.8044 - val_loss: 0.4945
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 82s 476ms/step - accuracy: 0.8079 - loss: 0.4928 - val_accuracy: 0.8044 - val_loss: 0.4950
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 82s 475ms/step - accuracy: 0.8075 - loss: 0.4920 - val_accuracy: 0.8044 - val_loss: 0.4946
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 82s 477ms/step - accuracy: 0.8049 - loss: 0.5004 - val_accuracy: 0.8044 - val_loss: 0.4955
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 80s 464ms/step - accuracy: 0.8074 - loss: 0.4939 - val_accuracy: 0.8044 - val_loss: 0.4950
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 87s 503ms/step - accuracy: 0.8067 - loss: 0.4936 - va

C:\Users\grego\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


172/172 ━━━━━━━━━━━━━━━━━━━━ 92s 506ms/step - accuracy: 0.7876 - loss: 1.3666 - val_accuracy: 0.7938 - val_loss: 0.5251
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 88s 510ms/step - accuracy: 0.7990 - loss: 0.5159 - val_accuracy: 0.7938 - val_loss: 0.5098
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 86s 501ms/step - accuracy: 0.7938 - loss: 0.5143 - val_accuracy: 0.7938 - val_loss: 0.5108
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 82s 479ms/step - accuracy: 0.7971 - loss: 0.5093 - val_accuracy: 0.7938 - val_loss: 0.5094
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 90s 526ms/step - accuracy: 0.7947 - loss: 0.5139 - val_accuracy: 0.7938 - val_loss: 0.5100
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 85s 494ms/step - accuracy: 0.7996 - loss: 0.5047 - val_accuracy: 0.7938 - val_loss: 0.5094
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 84s 491ms/step - accuracy: 0.7924 - loss: 0.5146 - val_accuracy: 0.7938 - val_loss: 0.5099
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 83s 481ms/step - accuracy: 0.7943 - loss: 0.5116 - val